In [52]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("ENTSOE_API_KEY")
import xml.etree.ElementTree as ET
import pandas as pd
import requests
url="https://web-api.tp.entsoe.eu/api"
zone = "10YAT-APG------L"
start = "202501222200"
end   = "202501282200"

In [55]:


##week ahead
params={
            "documentType": "A65",                   # Actual Total Load
            "processType": "A31",        # Actual Total Load
            "outBiddingZone_Domain": zone,
            "periodStart": start,
            "periodEnd": end,
            "securityToken":api_key
             # Actual Load
        }

response=requests.get(url=url,params=params)
response.status_code
root=ET.fromstring(response.content)

all_points = []

for ts in root.findall(".//{*}TimeSeries"):
    period = ts.find("{*}Period")

    # Period start
    period_start_str = period.find("{*}timeInterval/{*}start").text
    period_start = pd.to_datetime(period_start_str, utc=True)

    # HARD-CODED resolution = 1 day
    resolution = pd.Timedelta(days=1)

    for point in period.findall("{*}Point"):
        position = int(point.find("{*}position").text)
        value = float(point.find("{*}quantity").text)

        datetime_utc = period_start + (position - 1) * resolution

        all_points.append({
            "datetime_utc": datetime_utc,
            "load_MW": value
        })

df = (
    pd.DataFrame(all_points).sort_values("datetime_utc").reset_index(drop=True)
    
)

In [63]:
#day ahead_forcast
params={
            "documentType": "A65",                   # Actual Total Load
            "processType": "A01",        # Actual Total Load
            "outBiddingZone_Domain": zone,
            "periodStart": start,
            "periodEnd": end,
            "securityToken":api_key
             # Actual Load
        }

response=requests.get(url=url,params=params)
response.status_code
root=ET.fromstring(response.content)

all_points = []

for ts in root.findall(".//{*}TimeSeries"):
    period = ts.find("{*}Period")
    period_start_str = period.find("{*}timeInterval/{*}start").text
    period_start = pd.to_datetime(period_start_str)

    resolution_str = period.find("{*}resolution").text
    # if resolution_str=="PT60M:":
    minutes = int(resolution_str[2:-1])  # PT60M -> 60, PT15M -> 15

    for point in period.findall("{*}Point"):
        position = int(point.find("{*}position").text)
        value = float(point.find("{*}quantity").text)
        timestamp = period_start + pd.Timedelta(minutes=(position - 1) * minutes)

        all_points.append({
                    "datetime_utc": timestamp,
                    "load_MW": value
                })

        
    df = pd.DataFrame(all_points)
    df = df.sort_values("datetime_utc").reset_index(drop=True)
        
                
        

In [ ]:
#Installed generation capacity
# ENTSO-E Installed Generation per Type (A68) Year Ahead (A33)
# for specific PsrType

params = {
    "documentType": "A68",       # Installed generation per type
    "processType": "A33",        # Year ahead
    "in_Domain": "10YBE----------2",  # Example Control Area / Bidding Zone
    "periodStart": "202112312300",   # yyyyMMddHHmm
    "periodEnd": "202212312300",     # yyyyMMddHHmm
      # Wind Onshore, Wind Offshore, Solar, Fossil Hard Coal
    "securityToken":api_key
    
}


response=requests.get(url=url,params=params)
response.status_code
root=ET.fromstring(response.content)
response.status_code
target_psr = ["B19", "B18", "B16", "B05"]  # Wind Onshore, Wind Offshore, Solar, Fossil Hard Coal

all_data = []

# Iterate over TimeSeries
for ts in root.findall(".//{*}TimeSeries"):
    psr = ts.find(".//{*}MktPSRType/{*}psrType").text
    if psr not in target_psr:
        continue

    period = ts.find("{*}Period")
    start_time = pd.to_datetime(period.find("{*}timeInterval/{*}start").text)

    # For annual data, we just take the start of the period
    for point in period.findall("{*}Point"):
        quantity = float(point.find("{*}quantity").text)
        all_data.append({
            "year_start": start_time,
            "psrType": psr,
            "quantity_MW": quantity
        })

# Convert to DataFrame
df = pd.DataFrame(all_data)
print(df)



200

In [111]:

#Installed generation capacity
# ENTSO-E Installed Generation per Type (A68) Year Ahead (A33)
# for specific PsrType

params = {
    "documentType": "A71",       # Installed generation per type
    "processType": "A33",        # Year ahead
    "in_Domain": "10YDE-VE-------2",  # Example Control Area / Bidding Zone
    "periodStart": "202308010000",   # yyyyMMddHHmm
    "periodEnd": "202308010000",     # yyyyMMddHHmm
      # Wind Onshore, Wind Offshore, Solar, Fossil Hard Coal
    "securityToken":api_key
    
}


response=requests.get(url=url,params=params)
response.status_code
root=ET.fromstring(response.content)
response.status_code
 # Wind Onshore, Wind Offshore, Solar, Fossil Hard Coal




200

In [112]:
ns = {'ns': root.tag.split('}')[0].strip('{')}

all_data = []

for ts in root.findall('ns:TimeSeries', ns):
    series_id = ts.find('ns:mRID', ns).text
    series_name = ts.find('ns:registeredResource.name', ns).text
    unit = ts.find('ns:quantity_Measure_Unit.name', ns).text

    for period in ts.findall('ns:Period', ns):
        # start_time = period.find('ns:timeInterval/ns:start', ns).text
        # end_time = period.find('ns:timeInterval/ns:end', ns).text

        for point in period.findall('ns:Point', ns):
            quantity = float(point.find('ns:quantity', ns).text)
            all_data.append({
                'SeriesID': series_id,
                'Name': series_name,
                'Unit': unit,
                # 'Start': start_time,
                # 'End': end_time,
                'Quantity': quantity
            })

df = pd.DataFrame(all_data)
print(df.head(10))

           SeriesID                     Name Unit  Quantity
0  5a11a1e5b0544181   HKW Halle Dieselstraße  MAW     162.0
1  2fb07af54f3a4dfa              GuD Marzahn  MAW     264.0
2  dd757e96e5634b63                   Arkona  MAW     378.0
3  83d8168326d54cfd         UW Bertikow Wind  MAW     475.0
4  cff681339ad74bab                 HKW Nord  MAW     167.0
5  78208bb6bfb04bef            SP Gottesgabe  MAW     122.2
6  eb297d702fd345f1            SP Alttrebbin  MAW     122.2
7  7426aa7f349f42d3            Arcadis Ost 1  MAW     257.0
8  5a4dc7d39b43476b    PV Boitzenburger Land  MAW     180.3
9  037f16e6ceab4591  Windpark Baltic 1 and 2  MAW     336.3


<Element '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}GL_MarketDocument' at 0x0000024EE74A2890>

{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}GL_MarketDocument
[<Element '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}mRID' at 0x0000024EE7278180>, <Element '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}type' at 0x0000024EE7278130>, <Element '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}process.processType' at 0x0000024EE7278450>, <Element '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}sender_MarketParticipant.mRID' at 0x0000024EE72786D0>, <Element '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}sender_MarketParticipant.marketRole.type' at 0x0000024EE72791C0>]


In [103]:

# Class A
class A:
    def __init__(self, a_param):
        self.a_param = a_param
    
    def func_a(self):
        return f"Function A: a_param = {self.a_param}"

# Class B which uses Class A
class B:
    def __init__(self, a_obj, b_param):
        self.a_obj = a_obj  # store the A object
        self.b_param = b_param
    
    def func_b(self):
        return f"Function B: b_param = {self.b_param}"

    # optional: use A's function from B
    def use_a_from_b(self):
        return self.a_obj.func_a()

# Class C which uses Class B
class C:
    def __init__(self, a_param_for_A, c_param):
        # Create an A object inside C
        a_instance = A(a_param_for_A)
        # Create a B object inside C, passing the A object
        self.b_obj = B(a_instance, b_param="B's default")
        self.c_param = c_param
    
    def func_c(self):
        return f"Function C: c_param = {self.c_param}"

    # Access A's function through B
    def use_a_from_c(self):
        return self.b_obj.a_obj.func_a()

    # Access B's function through C
    def use_b_from_c(self):
        return self.b_obj.func_b()


# ================================
# Example usage
obj_c = C(a_param_for_A="Hello A", c_param="Hello C")

print(obj_c.func_c())          # Function C
print(obj_c.use_b_from_c())    # Function B
print(obj_c.use_a_from_c())    # Function A


Function C: c_param = Hello C
Function B: b_param = B's default
Function A: a_param = Hello A


In [106]:
df["Quantity"].sum()

10951.899999999998